## Recursive Feature Selection using Random Forests importance

Random Forests assign equal or similar importance to features that are highly correlated. In addition, when features are correlated, the importance assigned is lower than the importance attributed to the feature itself, should the tree be built without the correlated counterparts.

Therefore, instead of eliminating features based on importance by brute force like we did in the previous notebook by using SelectFromModel, we could get a better selection by removing one feature at a time, and recalculating the importance on each round. This procedure is called Recursive Feature Elimination (RFE)

RFE is a hybrid between embedded and wrapper methods: it is based on computation derived when fitting the model, but it also requires fitting several models.

The cycle is as follows:

- Build Random Forests using all features
- Remove least important feature
- Build Random Forests and recalculate importance on the remaining feature sets
- Repeat until a criteria is met

In this situation, when a feature that is highly correlated to another one is removed, then, the importance of the remaining feature increases. This may lead to a better feature space selection. On the downside, building several Random Forests is quite time and compute resource consuming, in particular if the dataset contains a high number of features.

I will demonstrate how to select features based Random Forests importance recursively using sklearn on a classification dataset.

In [39]:
# import libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

from sklearn.metrics import roc_auc_score

In [28]:
# load dataset

data = pd.read_csv('../precleaned-datasets/dataset_2.csv')
data.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.532710,3.280834,17.982476,4.404259,2.349910,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417
1,5.821374,12.098722,13.309151,4.125599,1.045386,1.832035,1.833494,0.709090,8.652883,0.102757,...,2.479789,7.795290,3.557890,17.383378,15.193423,8.263673,1.878108,0.567939,1.018818,1.416433
2,1.938776,7.952752,0.972671,3.459267,1.935782,0.621463,2.338139,0.344948,9.937850,11.691283,...,1.861487,6.130886,3.401064,15.850471,14.620599,6.849776,1.098210,1.959183,1.575493,1.857893
3,6.020690,9.900544,17.869637,4.366715,1.973693,2.026012,2.853025,0.674847,11.816859,0.011151,...,1.340944,7.240058,2.417235,15.194609,13.553772,7.229971,0.835158,2.234482,0.946170,2.700606
4,3.909506,10.576516,0.934191,3.419572,1.871438,3.340811,1.868282,0.439865,13.585620,1.153366,...,2.738095,6.565509,4.341414,15.893832,11.929787,6.954033,1.853364,0.511027,2.599562,0.811364


In [29]:
data.shape

(50000, 109)

In [30]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 108), (15000, 108))

## Select features recursively

In [31]:
# we do model training and feature selection in 2 lines of code

# first I specify the Random Forest and its parameters

# Then RFE from sklearn to remove features recursively

# RFE will remove one feature at each iteration => the least  important.
# then it will build another random forest and repeat
# till a criteria is met.

# in sklearn the criteria to stop is an arbitrary number
# of features to select, that we need to decide before hand
# not the best solution, but a solution
rf = RandomForestClassifier(n_estimators=10, n_jobs=2, random_state=10)

sel_ = RFE(rf, n_features_to_select=27)

sel_.fit(X_train, y_train)

RFE(estimator=RandomForestClassifier(n_estimators=10, n_jobs=2,
                                     random_state=10),
    n_features_to_select=27)

In [32]:
sel_.estimator_.feature_importances_

array([0.03347331, 0.03250509, 0.03412022, 0.03469516, 0.03930595,
       0.03474728, 0.03551822, 0.03330548, 0.03595636, 0.03517261,
       0.03280437, 0.09744573, 0.03318106, 0.03393011, 0.03509391,
       0.03275715, 0.03274527, 0.03385682, 0.03153768, 0.03546755,
       0.03286689, 0.03546021, 0.03292808, 0.0332371 , 0.03757222,
       0.03336266, 0.04695349])

In [33]:
# this command let's me visualise those features that were selected.

# sklearn will select those features which importance values
# are greater than the mean of all the coefficients.

sel_.get_support()

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True, False, False, False,  True, False, False,
       False, False,  True,  True, False, False,  True, False, False,
       False,  True, False, False,  True, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True, False,  True,
       False, False, False, False, False,  True, False, False,  True,
       False, False, False,  True,  True,  True, False,  True, False,
        True,  True, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False,  True, False])

In [34]:
# let's make a list and count the selected features

selected_feats = X_train.columns[sel_.get_support()]
selected_feats

Index(['var_4', 'var_14', 'var_21', 'var_25', 'var_30', 'var_31', 'var_34',
       'var_38', 'var_41', 'var_46', 'var_53', 'var_55', 'var_56', 'var_70',
       'var_71', 'var_73', 'var_79', 'var_82', 'var_86', 'var_87', 'var_88',
       'var_90', 'var_92', 'var_93', 'var_96', 'var_104', 'var_108'],
      dtype='object')

In [35]:
len(selected_feats)

27

In [36]:
# these are the features selected in the previous
# notebook where we used SelectFromModel from sklearn
# without doing it recursively

previous_lecture_selected_features = [
    'var_1', 'var_2', 'var_6', 'var_9', 'var_13', 'var_15', 'var_16', 'var_17',
    'var_20', 'var_21', 'var_30', 'var_34', 'var_37', 'var_55', 'var_60',
    'var_67', 'var_69', 'var_70', 'var_71', 'var_82', 'var_87', 'var_88',
    'var_95', 'var_96', 'var_99', 'var_103', 'var_108'
]

### Compare performance of feature subsets

In [37]:
# create a function to build random forests and compare
# their performance in train and test sets

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [42]:
# features selected recursively
run_randomForests(X_train[selected_feats],
                  X_test[selected_feats],
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7046591498448564
Test set
Random Forests roc-auc: 0.6904472637540937


In [41]:
# features selected recursively
run_randomForests(X_train[previous_lecture_selected_features],
                  X_test[previous_lecture_selected_features],
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7126509093226299
Test set
Random Forests roc-auc: 0.7009195513033531


We see that RFE did not return a better subset of features. The performance of the model built using the variables selected directly from the first RandomForest was enough to find a good subset of features.

In my opinion the RFE from sklearn does not bring forward a massive advantage respect to the SelectFromModel method and personally I tend to use the second to select my features.
